In [ ]:
import psycopg2 as p
from datetime import datetime

In [2]:
import psycopg2 as p
from datetime import datetime
password = '*****'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

# Step 1: deactivations based on missing census_nums or inactive DOTs in the MCMIS file

#### Still need to figure out if private fleets will fall into this deactivation

## OO Deactivation

In [ ]:
# Based on missing census_num in MCMIS
# Automatically happens
cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'N',
    update_date = now()::date

from fleetseek.fleet as f
left join mcmis as m
on f.usdot = m.census_num
where 
    m.census_num is null and
    f.active_flag = 'Y' and 
    f.product_code = 'OOD'
""")
conn.commit()

In [ ]:
# Outdated mcs150_date

cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'N',
    update_date = now()::date

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
where 
    m.mcs150_date < now() - interval '30 months' and
    f.product_code = 'OOD' and
    f.active_flag = 'Y'

""")

## FH Deactivation


In [ ]:
# For FH fleets between 1-19 power units based on missing census_num in MCMIS
cur.execute("""
update fleetseek.fleet
set active_flag = 'N'
from fleetseek.fleet as f
left join mcmis as m
on f.usdot = m.census_num
where 
    m.census_num is null and
    f.active_flag = 'Y' and 
    f.product_code = 'NMC' and
    f.total_vehicles between 1 and 19 and
    f.usdot != 0
""")
conn.commit()

In [ ]:
# FH fleets with old mcs150_date

cur.execute("""
update fleetseek.fleet
set active_flag = 'N'

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
where
    f.product_code = 'NMC' and
    f.total_vehicles between 1 and 19 and
    f.usdot != 0 and
    m.mcs150_date < now() - interval '30 months'

""")

# Step 2: Reactivations

## OO Reinstate

In [3]:
#12/19/2019 first pass

# 1/23/2020 additions. Reinstate based on new mcs150_dates, will also capture DOTs. This will also automatically update fleet numbers for OO. 
# As of 1/27/2020 it takes less than 1 minute to update

cur.execute("""
update fleetseek.fleet
set 
    trucks_owned = sub_q.owntruck,
    tractors_owned = sub_q.owntract,
    trailers_owned = sub_q.owntrail,
    trucks_leased = sub_q.trucks_leased,
    tractors_leased = sub_q.tractors_leased,
    trailers_leased = sub_q.trailers_leased,
    total_trucks = sub_q.total_trucks,
    total_tractors = sub_q.total_tractors,
    total_trailers = sub_q.total_trailers,
    total_vehicles = sub_q.total_vehicles,
    total_vehicles_owned = sub_q.total_vehicles_owned,
    total_vehicles_leased = sub_q.total_vehicles_leased,
    dot_hmind = sub_q.hm_ind,
    year_mileage = sub_q.mcs150mileageyear,
    vehicle_miles = sub_q.mlg150,
    drivers = sub_q.tot_drs,
    cdl_drivers = sub_q.cdl_drivers,
    active_flag = 'Y'
    
from 
(
    select 
        census_num, name, owntruck, owntract, owntrail, sum(trptruck+trmtruck) as trucks_leased, sum(trptract+trmtract) as tractors_leased, 
        sum(trptrail+trmtrail) as trailers_leased, sum(owntruck+trptruck+trmtruck) as total_trucks, sum(owntract+trmtract+trptract) as total_tractors,
        sum(owntrail+trmtrail+trptrail) as total_trailers, sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
        sum(owntruck+owntract) as total_vehicles_owned, sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased, hm_ind, mcs150mileageyear, 
        mlg150, tot_drs, cdl_drs as cdl_drivers, mcs150_date, passengers, class, carship
    from mcmis
    group by census_num, name, owntruck, owntract, owntrail, hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, mcs150_date, passengers, class, carship
) as sub_q

where
    fleet.usdot = sub_q.census_num and
    fleet.active_flag = 'N' and
    fleet.update_date < sub_q.mcs150_date and
    sub_q.mcs150_date > now() - interval '2 years' and
    sub_q.total_tractors = 1 and
    sub_q.total_trailers <= 1 and
    sub_q.passengers is null and
    (sub_q.class similar to 'A%|B%' or
    sub_q.class = 'AB') and
    sub_q.carship like 'C%' and
    fleet.product_code = 'OOD'
""")
conn.commit()

## FH reinstate

In [ ]:
# For-hire reinstatement based on original rules

cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'Y'
from 
(
    select 
        census_num, name, owntruck, owntract, owntrail, sum(trptruck+trmtruck) as trucks_leased, sum(trptract+trmtract) as tractors_leased, 
        sum(trptrail+trmtrail) as trailers_leased, sum(owntruck+trptruck+trmtruck) as total_trucks, sum(owntract+trmtract+trptract) as total_tractors,
        sum(owntrail+trmtrail+trptrail) as total_trailers, sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
        sum(owntruck+owntract) as total_vehicles_owned, sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased, hm_ind, mcs150mileageyear, 
        mlg150, tot_drs, cdl_drs as cdl_drivers, mcs150_date, passengers, class, carship
    from mcmis
    group by census_num, name, owntruck, owntract, owntrail, hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, mcs150_date, passengers, class, carship
) as sub_q
where 
    fleet.usdot = sub_q.census_num and
    fleet.active_flag = 'N'
    fleet.update_date < sub_q.mcs150_date and
    sub_q.mcs150_date > now() - interval '2 years' and
    
        (sub_q.total_trucks >= 2 or 
        (sub_q.total_trucks = 1 and sub_q.total_tractors = 1) or
        (sub_q.total_trucks >= 1 and sub_q.total_tractors = 0 and sub_q.total_trailers >= 2) or
        (sub_q.total_trucks = 0 and sub_q.total_tractors >= 1 and sub_q.total_trailers >=2)) and
        
    sub_q.class in ('A', 'AB', 'ABD', 'ABG', 'ABL', 'AG', 'AGL', 'AL', 'B', 'BL') and
    fleet.product_code = 'NMC'

""")

# Step 3a: All changes to fleets sizes. This will happen after activations/deactivations for fleet sizes from 1 to 19



## Fleet table and fleet_primary

In [2]:
# The suppress_redundant_updates_trigger will prevent any repeat updates that don't actually change the rows
# Updates both fleet and fleet_primary

# 1/21/2020 first pass
cur.execute("""
with updated as (

    update fleetseek.fleet
    set 

        trucks_owned = sub_q.owntruck,
        tractors_owned = sub_q.owntract,
        trailers_owned = sub_q.owntrail,
        trucks_leased = sub_q.trucks_leased,
        tractors_leased = sub_q.tractors_leased,
        trailers_leased = sub_q.trailers_leased,
        total_trucks = sub_q.total_trucks,
        total_tractors = sub_q.total_tractors,
        total_trailers = sub_q.total_trailers,
        total_vehicles = sub_q.total_vehicles,
        total_vehicles_owned = sub_q.total_vehicles_owned,
        total_vehicles_leased = sub_q.total_vehicles_leased,
        dot_hmind = sub_q.hm_ind,
        year_mileage = sub_q.mcs150mileageyear,
        vehicle_miles = sub_q.mlg150,
        drivers = sub_q.tot_drs,
        cdl_drivers = sub_q.cdl_drivers,
        update_date = now()::date
    from 
        (select 
            census_num, 
            name, 
            owntruck, 
            owntract, 
            owntrail, 
            sum(trptruck+trmtruck) as trucks_leased, 
            sum(trptract+trmtract) as tractors_leased, 
            sum(trptrail+trmtrail) as trailers_leased, 
            sum(owntruck+trptruck+trmtruck) as total_trucks, 
            sum(owntract+trmtract+trptract) as total_tractors,
            sum(owntrail+trmtrail+trptrail) as total_trailers, 
            sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
            sum(owntruck+owntract) as total_vehicles_owned, 
            sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased, 
            hm_ind, 
            mcs150mileageyear, 
            mlg150, 
            tot_drs, 
            cdl_drs as cdl_drivers, 
            mcs150_date, 
            passengers, 
            class, 
            carship

         from mcmis
         group by owntruck, owntract, owntrail, mcs150_date, census_num, name, hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, passengers, class, carship
        ) as sub_q

    where fleet.usdot = sub_q.census_num and
    fleet.update_date < sub_q.mcs150_date and
    sub_q.mcs150_date > now() - interval '2 years' and
    fleet.active_flag = 'Y' and
    sub_q.total_vehicles between 1 and 19 and
    fleet.product_code in ('NMC', 'PFD')
    returning
        fleet.fleet_id,
        fleet.total_vehicles,
        fleet.total_trailers

)

update fleetseek.fleet_primary
set
    total_vehicles = updated.total_vehicles,
    total_trailers = updated.total_trailers,
    update_date = now()::date

from updated
where fleet_primary.fleet_id = updated.fleet_id


""")
conn.commit()

# Step 3b: deactivating fleets if they fall below thresholds

In [ ]:
# Deactivating OO and NMC, happens before sending new addresses to Business Coder
cur.execute("""
update fleetseek.fleet
    set active_flag = 'N'
where product_code in ('OOD', 'NMC') and
total_tractors = 0 and
total_trailers = 0 and
total_trucks = 1 and
active_flag = 'Y'
""")
conn.commit()

## Private Fleet deactivation (1/27/2020 still holding off on this)

In [ ]:
#For PFD companies
cur.execute("""
update fleetseek.fleet
set active_flag = 'N'
from fleetseek.fleet as f
where 
    f.product_code = 'PFD' and
    f.total_tractors = 1 and
    f.total_trailers = 0 and
    f.total_trucks <= 1 and 
    f.total_vehicles <= 1 and
    f.active_flag = 'Y'
""")
conn.commit()

# Step 3c: inserting to be sent to the Business Coder API

#### Will be for both auto and manual updates


In [ ]:
# 1/21/2020 Initial write

# Won't be able to do Canadian or Mexican addresses since Business Coder only works for US addresses

cur.execute("""
insert into fleetseek.updates_bn_coder_output 

select f.fleet_id, f.usdot, m.name, m.phy_str, m.mai_str, m.tel_num, 

    case
        when (upper(left(replace(replace(replace(replace(l.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.phy_str, '-', ''), '&', ''), ' ', ''), 3)) and 
              upper(left(replace(replace(replace(replace(l2.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.mai_str, '-', ''), '&', ''), ' ', ''), 3))) then 'both'
              
        when (upper(left(replace(replace(replace(replace(l.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.phy_str, '-', ''), '&', ''), ' ', ''), 3)) and 
              upper(left(replace(replace(replace(replace(l2.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) = upper(left(replace(replace(replace(m.mai_str, '-', ''), '&', ''), ' ', ''), 3))) then 'physical'
        
        when (upper(left(replace(replace(replace(replace(l.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) = upper(left(replace(replace(replace(m.phy_str, '-', ''), '&', ''), ' ', ''), 3)) and 
              upper(left(replace(replace(replace(replace(l2.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.mai_str, '-', ''), '&', ''), ' ', ''), 3))) then 'mailing'
    end as differences,
    
    case
        when (m.phy_str = m.mai_str) then true else false
    end as uniform_mcmis_addresses
    
from mcmis as m
join fleetseek.fleet as f
on m.census_num = f.usdot 
join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
join fleetseek.location as l
on fp.primary_physical_location_id = l.location_id 
join fleetseek.location as l2
on fp.primary_mailing_location_id = l2.location_id
where
    (upper(left(replace(replace(replace(replace(l.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.phy_str, '-', ''), '&', ''), ' ', ''), 3)) or 
     upper(left(replace(replace(replace(replace(l2.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.mai_str, '-', ''), '&', ''), ' ', ''), 3))) and 
     m.mcs150_date > f.update_date and
     f.active_flag = 'Y' and
     f.product_code != 'CMC' and
     f.product_code != 'MMC'

on conflict (fs_fleet_id) do nothing

""")
conn.commit()

In [ ]:
# This cell will contain a command to run the new location information through the Business Coder API

### GVW Weight Class 8 update

In [ ]:
cur.execute("""
update fleetseek.fleet
set gvw_class_8 = 'Y'
from
    (select 
        census_num, owntract, trmtract, trptract, mcs150_date
      from mcmis
      group by census_num, owntract, trmtract, trptract, mcs150_date
      having sum(owntract+trmtract+trptract) >= 1
     ) as sub_q
where fleet.usdot = sub_q.census_num and
fleet.update_date < sub_q.mcs150_date and
fleet.total_vehicles between 1 and 19 and 
fleet.gvw_class_8 != 'Y' and
fleet.active_flag = 'Y'
""")
conn.commit()

### Fleet trailer type 

In [ ]:
# Changes to the utility types but not for fleets with a hazmat indicator

cur.execute("""
update fleetseek.fleet_trailer_type
set trailer_type_id = 142
from fleetseek.fleet_trailer_type as ftt
join fleetseek.fleet as f
on ftt.fleet_id = f.fleet_id
join (select 
      m.census_num, m.owntract, m.trmtract, m.trptract, m.owntrail, m.trmtrail, m.trptrail, m.mcs150_date
      from mcmis as m
      group by m.census_num, m.owntract, m.trmtract, m.trptract, m.owntrail, m.trmtrail, m.trptrail, m.mcs150_date
      having (sum(owntract+trmtract+trptract) = 0 and sum(owntrail+trmtrail+trptrail) > 0) 
     ) as sub_q
on f.usdot = sub_q.census_num
where f.update_date < sub_q.mcs150_date and
f.total_vehicles between 1 and 19 and
f.dot_hmind != 'Y' and
f.active_flag = 'Y'
""")
conn.commit()

# Step 4: changes in types after fleet size changes


### For-Hire to OO

In [ ]:
#This query will happen after all the deactivations and fleet size changes

cur.execute("""
update fleetseek.fleet
set 
    product_code = 'OOD',
    fleet_type_id = 105,
    sic_code = 4218
from (
    select 
        product_code as sub_q_product_code, fleet_type_id, sic_code, total_vehicles, total_tractors, total_trailers, total_trucks, fleet_id
    from fleetseek.fleet
    where total_tractors <= 1 and
    total_trailers <= 1 and
    total_vehicles <= 1 and
    total_trucks = 0 and
    product_code = 'NMC' and
    active_flag = 'Y'
    ) as sub_q
where fleet.fleet_id = sub_q.fleet_id and
product_code != 'OOD'
""")
conn.commit()

### OO to FH

In [ ]:
# 1/27/2020 still need to figure out how to update sic_code
cur.execute("""
update fleetseek.fleet
set 
    product_code = 'NMC'
    fleet_type_id = 104
    
from ( 
    select *
    from fleetseek.fleet as f
    join mcmis as m
    on f.usdot = m.census_num
    where 
        (f.total_trucks >= 2 or 
        (f.total_trucks = 1 and f.total_tractors = 1) or
        (f.total_trucks >= 1 and f.total_tractors = 0 and f.total_trailers >= 2) or
        (f.total_trucks = 0 and f.total_tractors >= 1 and f.total_trailers >=2)) and
        m.class in ('A', 'AB', 'ABD', 'ABG', 'ABL', 'AG', 'AGL', 'AL', 'B', 'BL') and
        f.product_code = 'OOD' and
        f.active_flag = 'Y'
) as sub_q
where fleet.fleet_id = sub_q.fleet_id and
product_code != 'NMC'

""")

# Step 5: changes to phone numbers, contacts, location info (different script, will happen after sending to API), etc.

### 1/24/2020 initial write

### Phone_number changes



In [ ]:
# Phone number changes for regular, toll-free, and fax
cur.execute("""
update fleetseek.phone_number
set
    phone_number = sub_q.tel_num
    updated_date_time = now()
from (
    select 
        m.tel_num, 
        pn.phone_number_id
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id
    where 
        m.tel_num != pn.phone_number and
        m.mcs150_date > f.update_date and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 1 and 19 and
        pn.phone_type_id = 1
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;


-- toll free update
update fleetseek.phone_number
set
    phone_number = sub_q.tel_num,
    updated_date_time = now()
from (
    select 
        m.tel_num, 
        pn.phone_number_id
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_toll_free_id = pn.phone_number_id
    where 
        m.tel_num != pn.phone_number and
        m.mcs150_date > f.update_date and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 1 and 19 and
        pn.phone_type_id = 2 and
        m.tel_num similar to ('800%|888%|877%|866%|855%|844%|833%')
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;

-- fax number update

update fleetseek.phone_number
set
    phone_number = sub_q.fax_num
    updated_date_time = now()
from (
    select 
        m.fax_num, 
        pn.phone_number,
        pn.phone_number_id,
        pn.phone_type_id,
        m.mcs150_date,
        f.update_date
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_fax_id = pn.phone_number_id
    where 
        m.fax_num != pn.phone_number and
        m.mcs150_date > f.update_date and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 1 and 19 and
        pn.phone_type_id = 3

) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id


""")
conn.commit()

## Email changes

In [ ]:
# No update date on this table
cur.execute("""
update fleetseek.email
    set email_address = sub_q.mcmis_email

from (

select lower(m.email_address) as mcmis_email, lower(e.email_address) as current_email, m.mcs150_date, f.update_date, e.email_id
from mcmis as m
join fleetseek.fleet as f
on m.census_num = f.usdot 
join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
join fleetseek.email as e
on fp.primary_email_id = e.email_id
where 
    lower(btrim(e.email_address)) != lower(btrim(m.email_address)) and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years' and
    f.active_flag = 'Y' and 
    f.total_vehicles between 1 and 19
) as sub_q
where email.email_id = sub_q.email_id

""")
conn.commit()

## Contact updates 

#### Contacts will not be deleted but rather different ones will be added

In [ ]:
cur.execute("""
alter table fleetseek.contact
add column mcmis_id
""")
conn.commit()

In [ ]:
# Go into contact and fleet_contact table

cur.execute("""
with inserted as 
(
    insert into fleetseek.contact (first_name, middle_name, last_name, suffix, mcmis_id)

    select 
        sub_q.first_name,
        sub_q.middle_name,
        sub_q.last_name,
        sub_q.suffix,
        sub_q.census_num
    from 
    (

        select 
        initcap(split_part(a.company_rep1a, ' ', 1)) as first_name,
        case
            when (a.count_of_spaces = 3 and (a.company_rep1a ilike '%III%' or a.company_rep1a ilike '%III')) then initcap(split_part(a.company_rep1a, ' ', 2))
            when (a.count_of_spaces = 3 and (a.company_rep1a not ilike '%jr' and a.company_rep1a not ilike '%jr.') and (a.company_rep1a not ilike '%sr' and a.company_rep1a not ilike '%sr.')) then initcap(concat(split_part(a.company_rep1a, ' ', 2), ' ',  split_part(a.company_rep1a, ' ', 3)))
            when (a.count_of_spaces = 3) then initcap(split_part(a.company_rep1a, ' ', 2))
            when (a.count_of_spaces = 2 and a.company_rep1a ilike '%III%') then null
            when (a.count_of_spaces = 2 and (a.company_rep1a not ilike '%jr' and a.company_rep1a not ilike '%jr.') and (a.company_rep1a not ilike '%sr' and a.company_rep1a not ilike '%sr.')) then initcap(split_part(a.company_rep1a, ' ', 2))
        end as middle_name,

        case 
            when (a.count_of_spaces = 3 and (a.company_rep1a ilike '%III%' or a.company_rep1a ilike '%III')) then initcap(split_part(a.company_rep1a, ' ', 3))
            when (a.count_of_spaces = 3 and (a.company_rep1a not ilike '%jr' and a.company_rep1a not ilike '%jr.') and (a.company_rep1a not ilike '%sr' and a.company_rep1a not ilike '%sr.')) then initcap(split_part(a.company_rep1a, ' ', 4))
            when (a.count_of_spaces = 1) then initcap(split_part(a.company_rep1a, ' ', 2)) 
            when (a.count_of_spaces = 2 and company_rep1a ilike '%III%') then initcap(split_part(a.company_rep1a, ' ', 2))
            when (a.count_of_spaces = 2 and (a.company_rep1a ilike '%jr%' or a.company_rep1a ilike '%sr%')) then initcap(split_part(a.company_rep1a, ' ', 2))
            when (a.count_of_spaces = 2 and (a.company_rep1a not ilike '%jr%' or a.company_rep1a not ilike '%sr%')) then initcap(split_part(a.company_rep1a, ' ', 3))
            when (a.count_of_spaces = 3) then initcap(split_part(a.company_rep1a, ' ', 3))
        end as last_name,

        case
            when (a.count_of_spaces = 3 and (a.company_rep1a ilike '%III%' or a.company_rep1a ilike '%III')) then split_part(a.company_rep1a, ' ', 4)
            when (a.count_of_spaces = 3 and (a.company_rep1a not ilike '%jr' and a.company_rep1a not ilike '%jr.') and (a.company_rep1a not ilike '%sr' and a.company_rep1a not ilike '%sr.')) then null
            when (a.count_of_spaces = 3) then split_part(a.company_rep1a, ' ', 4)
            when (a.count_of_spaces = 2 and a.company_rep1a ilike '%jr%') then split_part(a.company_rep1a, ' ', 3)
            when (a.count_of_spaces = 2 and a.company_rep1a ilike '%sr%') then split_part(a.company_rep1a, ' ', 3)
            when (a.count_of_spaces = 2 and a.company_rep1a ilike '%III%') then split_part(a.company_rep1a, ' ', 3)
        end as suffix,
        a.census_num,
        a.contact_id,
        a.company_rep1,
        a.company_rep2


        FROM (
            SELECT

                b.company_rep1_remove_space as company_rep1a,
                b.company_rep2_remove_space as company_rep2a,
                length(b.company_rep1_remove_space) - length(replace(b.company_rep1_remove_space,' ', '')) AS count_of_spaces,
                length(b.company_rep2_remove_space) - length(replace(b.company_rep2_remove_space, ' ', '')) as count_of_spaces2,
                b.census_num,
                b.contact_id,
                b.fleet_id,
                b.company_rep1,
                b.company_rep2


            FROM (
                    select 
                        m.census_num,
                        c.contact_id,
                        f.fleet_id,
                        m.company_rep1,
                        m.company_rep2,
                        replace(m.company_rep1, '  ', ' ') as company_rep1_remove_space,
                        replace(m.company_rep2, '  ', ' ') as company_rep2_remove_space
                    from mcmis as m
                    join fleetseek.fleet as f
                    on m.census_num = f.usdot
                    join fleetseek.fleet_primary as fp
                    on f.fleet_id = fp.fleet_id
                    join fleetseek.contact as c
                    on fp.primary_contact_id = c.contact_id
                 ) as b

             ) as a


    ) as sub_q
    join fleetseek.fleet as f
    on sub_q.census_num = f.usdot
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.contact as c
    on fp.primary_contact_id = c.contact_id
    where similarity(concat(upper(c. first_name), ' ', upper(c.middle_name), ' ', upper(c.last_name), ' ', upper(c.suffix)), sub_q.company_rep1) < 0.5 and
    f.total_vehicles between 1 and 19 and 
    f.active_flag = 'Y'

    returning 
        contact_id,
        mcmis_id
)

insert into fleetseek.fleet_contact (fleet_id, contact_id)
select
    mcmis_id,
    contact_id
from inserted;

alter table fleetseek.contact
drop column mcmis_id;

""")
conn.commit()